In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from collections import Counter


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data_filename = os.path.join(dirname, filenames[1])
df = pd.read_csv(data_filename)
df.head()

In [ ]:
df['choice_list'] = [x for x in df.drop(columns=['family_id','n_people']).to_numpy()]
flat_choice_list = [val for sublist in df['choice_list'].tolist() for val in sublist]
choice_day_freq = Counter(flat_choice_list)
inverse_choice_day_freq = {x:1/choice_day_freq[x] for x in range(1,101)}

In [ ]:
choice_fixed_cost = [0, 50, 50, 100, 200, 200, 300, 300, 400, 500, 500]
choice_var_cost = [0, 0, 9, 9, 9, 18, 18, 36, 36, 36+199, 36+398]
choice_cost = [[m,n] for m,n in zip(choice_fixed_cost,choice_var_cost)]

In [ ]:
df = df[['family_id','n_people','choice_list']]
df['fam_choice_cost'] = [[x[0]+m*x[1] for x in choice_cost] for m in df['n_people']]

### not weighted by choice order (higher value higher importance)
df['importance'] = [n*np.sum([inverse_choice_day_freq[x] for x in m]) for m,n in zip(df['choice_list'], df['n_people'])]

### weighted by choice order (lower value higher importance)
#df['importance'] = [np.sum([choice_day_freq[x]*(y+0.000001) for x,y in zip(m,n)]) for m,n in zip(df['choice_list'],df['fam_choice_cost'])]
df

In [ ]:
##### Testing Cell #############
daily_participant_array = np.random.randint(0,high=301,size=100)
occ = pd.DataFrame(daily_participant_array, columns=['day_participant'])
n_first_day = daily_participant_array[-1]
#occ['day_participant'] = daily_participant_array
occ['prev_day_participant'] = occ['day_participant'].shift(-1).fillna(n_first_day).astype(int)
occ['acc_cost'] = [((x - 125) / 400 ) * x ** (0.5 + (abs(x - y) / 50)) for x,y in zip(occ['day_participant'],occ['prev_day_participant'])]
out_of_range = occ[(occ['day_participant']<125) | (occ['day_participant']>300)]
occ.loc[(occ['day_participant']<125) | (occ['day_participant']>300),'acc_cost'] = [x*10**9 + abs(x-y)**7 for x,y in zip(out_of_range['day_participant'],out_of_range['prev_day_participant'])]
occ#['acc_cost'].sum()

In [ ]:
## Accounting Cost
# enumerate(daily_participant_array) => day, occupancy
def acc_cost(daily_participant_array):
    occ = pd.DataFrame(daily_participant_array, columns=['day_participant'])
    n_first_day = daily_participant_array[-1]
    occ['prev_day_participant'] = occ['day_participant'].shift(-1).fillna(n_first_day).astype(int)
    occ['acc_cost'] = [((x - 125) / 400 ) * x ** (0.5 + (abs(x - y) / 50)) for x,y in zip(occ['day_participant'],occ['prev_day_participant'])]
    hit_lb = occ[occ['day_participant']<125]
    hit_ub = occ[occ['day_participant']>300]
    occ.loc[occ['day_participant']<125,'acc_cost'] = [(125-x)*10**9 + abs(y-x)*10**5 for x,y in zip(hit_lb['day_participant'],hit_lb['prev_day_participant'])]
    occ.loc[occ['day_participant']>300,'acc_cost'] = [(x+300)*10**9 + abs(x-y)*10**6 for x,y in zip(hit_ub['day_participant'],hit_ub['prev_day_participant'])]
    #if occ['acc_cost'].median()>10**5:
    #    return occ['acc_cost'].median()
    #else:
    return occ['acc_cost'].sum()


In [ ]:
sorted_df = df.sort_values('importance', ascending=False)
sorted_df

In [ ]:
np.argmin(np.array([1,2]) + np.array([1,2]))

In [ ]:
## Greedy Search ############
daily_participant_array = np.zeros(100)
cum_pref_cost = 0
family_choices_given = np.zeros(5000) 
n_people = sorted_df.iloc[:,1].tolist()

for i in tqdm(range(5000)):
    choice_days = sorted_df.iloc[i,2]
    pref_costs = sorted_df.iloc[i,3][:-1]
    
    tmp_participant = 0
    acc_costs = np.zeros(10)
    for j in range(10):
        tmp_daily_participant_array = daily_participant_array.copy()
        tmp_daily_participant_array[choice_days[j]-1] += n_people[i]
        if tmp_daily_participant_array[choice_days[j]-1]>tmp_participant:
            tmp_participant = tmp_daily_participant_array[choice_days[j]-1]
            print(tmp_participant)
        acc_costs[j] = acc_cost(tmp_daily_participant_array)
        
    total_costs = pref_costs + acc_costs
    #print(total_costs)
    
    if tmp_participant<125:
        index_chosen_by_greedy = np.argmin(pref_costs)
    else:
        index_chosen_by_greedy = np.argmin(total_costs)
    cum_pref_cost += pref_costs[index_chosen_by_greedy]
    #print('this_family_total_cost: ',total_costs[index_chosen_by_greedy])
    print('cost_up_til_now: ', cum_pref_cost+acc_costs[index_chosen_by_greedy])
    
    family_choices_given[i] = index_chosen_by_greedy
    daily_participant_array[choice_days[index_chosen_by_greedy]-1] += n_people[i]
    #print(family_choices_given[i])
    print(daily_participant_array)

## Backup

In [ ]:
pref_cost_matrix = np.array([np.full((100),choice_cost[10][0]+choice_cost[10][1]*x) for x in family_choices_df['n_people']])
pref_cost_matrix
cost_matrix = [[n[0]+m*n[1] for m in range(2,9)] for n in choice_cost]
for i in range(5000):
    n_ppl = family_choices_df['n_people'][i]
    for j in range(10):
        day = family_choices_df['choice_list'][i][j]-1
        pref_cost_matrix[i][day] = cost_matrix[j][n_ppl-2]

# to call: acc_matrix[today,prev_day]
acc_matrix = np.zeros((176,176))
for i, x in enumerate(range(125,300+1)):
    for j, y in enumerate(range(125,300+1)):
        acc_matrix[i,j] = ((x - 125) / 400 ) * x ** (0.5 + (abs(x - y) / 50))

family_choices_df['pref_matrix'] = [x for x in pref_cost_matrix]


def total_cost(submission_array, family_choices_df, pref_cost_matrix, acc_matrix):
    # Preference Cost
    #data = family_choices_df.merge(submission_df, on='family_id')
    s = np.random.randint(0,high=1001)
    try:
        data = family_choices_df.copy()
        data['assigned_day'] = submission_array
        
        assert data['assigned_day'].max() <= 100
        assert data['assigned_day'].min() >= 1

        data['assigned_day'] = data['assigned_day'].round().astype(int)
        data['pref_cost'] = [x[int(y)-1] for x,y in zip(data['pref_matrix'],data['assigned_day'])]
        preference_cost = data['pref_cost'].sum()

        # Accounting cost
        daily_participant_df = data.groupby('assigned_day')['n_people'].sum().reset_index()
        assert daily_participant_df['n_people'].max() <= 300
        assert daily_participant_df['n_people'].min() >= 125
        

        day1 = daily_participant_df['n_people'].tolist()[-1]
        daily_participant_df['n_prev_day'] = daily_participant_df['n_people'].shift(-1).fillna(day1).astype(int)
        daily_participant_df['acc_cost'] = [acc_matrix[x-125][y-125] for x,y in zip(daily_participant_df['n_people'].astype(int),daily_participant_df['n_prev_day'].astype(int))]
        accounting_cost = daily_participant_df['acc_cost'].sum()
        res = preference_cost + accounting_cost
        print(res)
    except AssertionError:
        data['assigned_day'] = data['assigned_day'].round().astype(int)
        daily_participant_df = data.groupby('assigned_day')['n_people'].sum().reset_index()
        res = math.exp((data['assigned_day'].max() - data['assigned_day'].min())) * math.exp((daily_participant_df['n_people'].max()-daily_participant_df['n_people'].min()))
        if s==73:
            print(res)
    return res